In [1]:
#Import required libraries
import gspread
import sys
#from oauth2client.service_account import ServiceAccountCredentials
from google.auth.transport.requests import AuthorizedSession
from google.oauth2 import service_account
import pandas as pd
import numpy as np
import urllib
import sqlalchemy
from gspread_dataframe import set_with_dataframe
from gspread_dataframe import get_as_dataframe

In [2]:
#Import project specific functions
from column_map import column_map
from yesno_functions import *
from format_datetime import *

In [3]:
#Import shared functions
sys.path.append('..')
from IPM_Shared_Code_public.Python.google_creds_functions import create_assertion_session
from IPM_Shared_Code_public.Python.utils import get_config
from IPM_Shared_Code_public.Python.delta_functions import *
from IPM_Shared_Code_public.Python.sql_functions import *

C:\ProgramData\Anaconda3\lib\site-packages\authlib\client\__init__.py:12: AuthlibDeprecationWarning: Deprecate "authlib.client", USE "authlib.integrations.requests_client" instead.
It will be compatible before version 1.0.
Read more <https://git.io/Jeclj#file-rn-md>
  deprecate('Deprecate "authlib.client", USE "authlib.integrations.requests_client" instead.', '1.0', 'Jeclj', 'rn')


### Use the config file to setup connections

In [4]:
config = get_config('c:\Projects\config.ini')

driver = config['srv']['driver']
server = config['srv']['server']
dwh = config['db']['crowdsdb']
cred_file = config['google']['path_to_file']

In [5]:
con_string = 'Driver={' + driver + '};Server=' + server +';Database=' + dwh + ';Trusted_Connection=Yes;'
params = urllib.parse.quote_plus(con_string)
engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

### Execute the function to get the renamed columns for this sheet

In [6]:
#Call the column map function to get the dictionary to be used for renaming and subsetting the columns
col_rename = column_map('ambassador_dpr')

In [7]:
#Because of duplicate column names these columns are renamed based on the column index and the keys and 
#values need to be switched
col_rename = {v[0]: k for k, v in col_rename.items()}

In [8]:
cols = list(col_rename.values())

### Read the site reference list from SQL

In [9]:
sql = 'select * from crowdsdb.dbo.tbl_ref_sites'

In [10]:
site_ref = pd.read_sql(con = engine, sql = sql)[['site_id', 'site_desc', 'borough']]

### Read the current data from SQL

In [11]:
sql = 'select * from crowdsdb.dbo.tbl_dpr_ambassador'

In [12]:
ambass_sql = (pd.read_sql(con = engine, sql = sql)
              .drop(columns = ['ambassador_id', 'patroncount'])
              .fillna(value = np.nan, axis = 1))

In [13]:
sql_cols = list(ambass_sql.columns.values)

In [14]:
format_datetime(ambass_sql, 'encounter_timestamp')
format_datetime(ambass_sql, 'encounter_datetime')

In [15]:
float_cols = ['sd_pdcontact', 'closed_approach', 'closed_outcome', 'closed_pdcontact']
for c in float_cols:
    ambass_sql[c] = ambass_sql[c].astype(float)

In [16]:
hash_rows(ambass_sql, exclude_cols = ['site_id', 'encounter_timestamp'], hash_name = 'row_hash')

### Read the latest data from Google Sheets

In [17]:
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

In [ ]:
creds = service_account.Credentials.from_service_account_file(cred_file).with_scopes(scope)

In [ ]:
authed_session = AuthorizedSession(creds)

In [ ]:
client = gspread.Client(creds, authed_session)

In [18]:
sheet = client.open('_Combined INTERNAL PARKS Ambassador COVID-19 Reporting (Responses)')

In [19]:
ws = sheet.worksheet('Form Responses 1')

In [20]:
ambass = (get_as_dataframe(ws, evaluate_formulas = True, header= None)
          #Always remove row 0 with the column headers
          .iloc[1:]
          .rename(columns = col_rename)
          .fillna(value = np.nan, axis = 1))[cols]

In [21]:
ambass.head()

,encounter_timestamp,encounter_datetime,site_desc,location_adddesc,park_division,firstname_1,lastname_1,firstname_2,lastname_2,firstname_3,...,sd_amenity,sd_pdcontact,sd_comments,closed_amenity,closed_patroncount,closed_approach,closed_outcome,closed_pdcontact,closed_comments,borough
1,6/15/2020 8:25:09,6/15/2020 8:00:00,Williamsbridge Oval,NaN,Recreation,jeantheia,Hampton,NaN,NaN,NaN,...,Open field/multi-purpose play area,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bronx
2,6/15/2020 8:08:27,6/15/2020 15:00:00,St. Mary's Park,NaN,Recreation,jabor,forde,NaN,NaN,NaN,...,Bench/sitting area,NaN,there were over 200 people in the park,NaN,NaN,NaN,NaN,NaN,NaN,Bronx
3,6/14/2020 18:28:53,6/14/2020 18:30:00,Franklin D. Roosevelt Boardwalk and Beach,NaN,Park Administrator Ambassador,eric,means,sean,capers,toosdhi,...,Parking Lot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Staten Island
4,6/14/2020 18:27:33,6/14/2020 18:30:00,Franklin D. Roosevelt Boardwalk and Beach,NaN,Park Administrator Ambassador,Dane,Ambrose,Moses,Peters,NaN,...,Walking path,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Staten Island
5,6/14/2020 18:26:39,6/14/2020 18:30:00,Franklin D. Roosevelt Boardwalk and Beach,NaN,Park Administrator Ambassador,Waynette,Saunders,NaN,NaN,NaN,...,Baseball field,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Staten Island


In [22]:
yesno = ['sd_pdcontact', 'closed_approach', 'closed_outcome', 'closed_pdcontact']

In [23]:
yesno_cols(ambass, yesno)

In [24]:
#Remove rows with no timestamp because these rows have no data
ambass = ambass[ambass['encounter_timestamp'].notnull()]

In [25]:
ambass = ambass.merge(site_ref, how = 'left', on = ['site_desc', 'borough'])[sql_cols]

In [26]:
format_datetime(ambass, 'encounter_timestamp')
format_datetime(ambass, 'encounter_datetime')

In [27]:
ambass.head()

,encounter_timestamp,encounter_datetime,site_id,location_adddesc,park_division,firstname_1,lastname_1,firstname_2,lastname_2,firstname_3,...,sd_amenity,sd_pdcontact,sd_comments,closed_amenity,closed_patroncount,closed_approach,closed_outcome,closed_pdcontact,closed_comments,borough
0,06-15-2020 08:25:09,06-15-2020 08:00:00,X104,NaN,Recreation,jeantheia,Hampton,NaN,NaN,NaN,...,Open field/multi-purpose play area,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bronx
1,06-15-2020 08:08:27,06-15-2020 15:00:00,X045,NaN,Recreation,jabor,forde,NaN,NaN,NaN,...,Bench/sitting area,NaN,there were over 200 people in the park,NaN,NaN,NaN,NaN,NaN,NaN,Bronx
2,06-14-2020 18:28:53,06-14-2020 18:30:00,R046,NaN,Park Administrator Ambassador,eric,means,sean,capers,toosdhi,...,Parking Lot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Staten Island
3,06-14-2020 18:27:33,06-14-2020 18:30:00,R046,NaN,Park Administrator Ambassador,Dane,Ambrose,Moses,Peters,NaN,...,Walking path,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Staten Island
4,06-14-2020 18:26:39,06-14-2020 18:30:00,R046,NaN,Park Administrator Ambassador,Waynette,Saunders,NaN,NaN,NaN,...,Baseball field,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Staten Island


In [28]:
hash_rows(ambass, exclude_cols = ['site_id', 'encounter_timestamp'], hash_name = 'row_hash')

### Find the deltas based on the row hashes

In [29]:
ambass_deltas = (check_deltas(new_df = ambass, old_df = ambass_sql, on = ['encounter_timestamp', 'site_id'], 
                              hash_name = 'row_hash', dml_col = 'dml_verb'))[sql_cols + ['dml_verb']]

In [30]:
ambass_deltas.head()

,encounter_timestamp,encounter_datetime,site_id,location_adddesc,park_division,firstname_1,lastname_1,firstname_2,lastname_2,firstname_3,...,sd_pdcontact,sd_comments,closed_amenity,closed_patroncount,closed_approach,closed_outcome,closed_pdcontact,closed_comments,borough,dml_verb
0,06-15-2020 08:25:09,06-15-2020 08:00:00,X104,NaN,Recreation,jeantheia,Hampton,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bronx,I
1,06-15-2020 08:08:27,06-15-2020 15:00:00,X045,NaN,Recreation,jabor,forde,NaN,NaN,NaN,...,NaN,there were over 200 people in the park,NaN,NaN,NaN,NaN,NaN,NaN,Bronx,I
2,06-14-2020 18:28:53,06-14-2020 18:30:00,R046,NaN,Park Administrator Ambassador,eric,means,sean,capers,toosdhi,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Staten Island,I
3,06-14-2020 18:27:33,06-14-2020 18:30:00,R046,NaN,Park Administrator Ambassador,Dane,Ambrose,Moses,Peters,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Staten Island,I
4,06-14-2020 18:26:39,06-14-2020 18:30:00,R046,NaN,Park Administrator Ambassador,Waynette,Saunders,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Staten Island,I


In [31]:
ambass_inserts = ambass_deltas[ambass_deltas['dml_verb'] == 'I'][sql_cols]

In [32]:
ambass_inserts.head()

,encounter_timestamp,encounter_datetime,site_id,location_adddesc,park_division,firstname_1,lastname_1,firstname_2,lastname_2,firstname_3,...,sd_amenity,sd_pdcontact,sd_comments,closed_amenity,closed_patroncount,closed_approach,closed_outcome,closed_pdcontact,closed_comments,borough
0,06-15-2020 08:25:09,06-15-2020 08:00:00,X104,NaN,Recreation,jeantheia,Hampton,NaN,NaN,NaN,...,Open field/multi-purpose play area,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bronx
1,06-15-2020 08:08:27,06-15-2020 15:00:00,X045,NaN,Recreation,jabor,forde,NaN,NaN,NaN,...,Bench/sitting area,NaN,there were over 200 people in the park,NaN,NaN,NaN,NaN,NaN,NaN,Bronx
2,06-14-2020 18:28:53,06-14-2020 18:30:00,R046,NaN,Park Administrator Ambassador,eric,means,sean,capers,toosdhi,...,Parking Lot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Staten Island
3,06-14-2020 18:27:33,06-14-2020 18:30:00,R046,NaN,Park Administrator Ambassador,Dane,Ambrose,Moses,Peters,NaN,...,Walking path,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Staten Island
4,06-14-2020 18:26:39,06-14-2020 18:30:00,R046,NaN,Park Administrator Ambassador,Waynette,Saunders,NaN,NaN,NaN,...,Baseball field,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Staten Island


In [33]:
ambass_inserts.to_sql('tbl_dpr_ambassador', engine, index = False, if_exists = 'append')

In [34]:
ambass_updates = ambass_deltas[ambass_deltas['dml_verb'] == 'U'][sql_cols]

In [35]:
ambass_updates.head()

,encounter_timestamp,encounter_datetime,site_id,location_adddesc,park_division,firstname_1,lastname_1,firstname_2,lastname_2,firstname_3,...,sd_amenity,sd_pdcontact,sd_comments,closed_amenity,closed_patroncount,closed_approach,closed_outcome,closed_pdcontact,closed_comments,borough


In [36]:
sql_update(ambass_updates, 'tbl_dpr_ambassador', engine, ['encounter_timestamp', 'site_id'])